## Gather Follower Descriptions
We start with a twitter user name (as well as things like twitter credentials) and pull all the followers of that user. We then pull the descriptions of those users and write them to a file for future mining. You can use more than one user in a group.

I wrote some functions for you in the file `twitter_functions.py`. This will need to be in the same folder where you're running this notebook. 

In [1]:
from datetime import datetime
import json
from twitter_functions import * # these are the functions I wrote for you.

The next cell holds your Twitter authorization credentials. Then it calls a function that initializes your connection to Twitter. I've left my keys in there (slightly perturbed so that they won't actually work) so that you see the form these take).

In [2]:
auth =  { "consumer_key": "zNf902L1N3KJ0KICsJZPs4FxS",
          "consumer_secret": "MvKCr3x2Z2VM1Tekzq7mMdnxqMzc6BUNgRkyvQXNp0OjuZ7KwH",
          "access_key": "922636098081234944-xnes1B7sbaLJJXAZ96xmZNLg0YvARzo",
          "access_secret": "BqZvcKvMYLqMGhlqO4eGiyR1F5w5i1Pt5RbvWxDozVYI9"
        }

api = initialize_twitter(auth)

Now you set the handle (or handles) that represent one group or topic on Twitter. These should be in a list. The output file name (`ofile_name`) is determined based on today's date and the first element in the list. Feel free to modify. 

In [3]:
#starting_user = ['HealthRanger'] # my first group
starting_user = ['TannersDad'] # my second group


ofile_name = (datetime.today().strftime("%Y%m%d") + "_" + 
             starting_user[0] + "_" + # Just take the first one if there are multiple
             "followers.txt")

In [4]:
# We'll now go lookup the full information on your starting user(s). 
starting_user_id = []

# All records will be a dictionary with the twitter ID as the key and 
# a UserRecord as the value. This is a named tuple I created. 
all_records = lookup_users_from_handles(api, starting_user)

# We need the IDs that we're getting followers from in a list. 
for id in all_records : #access the keys, which are ids.
    starting_user_id.append(id)

Start lookup_users_from_ids on 1 handles.
20171213-185255: looking up user records for 1 handles.
20171213-185256:  users pulled:  1
total failures: 0


In [5]:
# How long is it going to take us to pull these followers?
total_followers = 0
for id, rec in all_records.items() :
    total_followers += rec.followers_count
    
print("A complete run with no limits run is " + 
      "going to take {min:.2f} minutes ({hour:.2f} hours)".format(min=total_followers/5000,
                                                                              hour=total_followers/(60*5000)))

A complete run with no limits run is going to take 4.31 minutes (0.07 hours)


In [6]:
# Now let's pull all the followers of our starting_user
# the function I wrote allows you to cap the number of followers you pull
# and uses the ID to generate the query.
# 
# Note that this pull is subject to rate limiting. You can make 15 calls per
# 15 minute window and each can return 5000 users. 
followers_of_starting = gather_followers(api,
                                         starting_user_id,
                                         follower_limit=None) # Modify this limit if you need to. Set it to "None" to get all   

# followers_of_starting will be a dictionary with the key being the id(s) in starting_user_id
# and the value is a list of all the followers.

Pulling followers for 16854311
Number pulled: 5000
Number pulled: 10000
Number pulled: 15000
Number pulled: 20000
Number pulled: 21533


In [7]:
# And now we'll "hydrate" these user records. 
# Note that this is going to print a lot of rows to the screen, 
# partially just so you can know it's working.  
for start_id, list_of_followers in followers_of_starting.items() :
    
    # Using a set here instead of a list so that we pull each ID only once.
    ids_to_hydrate = {id for id in list_of_followers if id not in all_records} 
    
    these_records = lookup_users_from_ids(api,ids=ids_to_hydrate)

    for id, rec in these_records.items() :
        all_records[id] = rec


Start lookup_users_from_ids on 21533 IDs.
20171213-185309: looking up user records for 100 IDs.
20171213-185310: looking up user records for 100 IDs.
20171213-185311: looking up user records for 100 IDs.
20171213-185312: looking up user records for 100 IDs.
20171213-185314: looking up user records for 100 IDs.
20171213-185315: looking up user records for 100 IDs.
20171213-185316: looking up user records for 100 IDs.
20171213-185317: looking up user records for 100 IDs.
20171213-185318: looking up user records for 100 IDs.
20171213-185319: looking up user records for 100 IDs.
20171213-185320: looking up user records for 100 IDs.
20171213-185321: looking up user records for 100 IDs.
20171213-185323: looking up user records for 100 IDs.
20171213-185324: looking up user records for 100 IDs.
20171213-185325: looking up user records for 100 IDs.
20171213-185326: looking up user records for 100 IDs.
20171213-185327: looking up user records for 100 IDs.
20171213-185328: looking up user records

20171213-185615: looking up user records for 100 IDs.
20171213-185616: looking up user records for 100 IDs.
20171213-185618: looking up user records for 100 IDs.
20171213-185618: looking up user records for 100 IDs.
20171213-185620: looking up user records for 100 IDs.
20171213-185621: looking up user records for 100 IDs.
20171213-185623: looking up user records for 100 IDs.
20171213-185624: looking up user records for 100 IDs.
20171213-185625: looking up user records for 100 IDs.
20171213-185626: looking up user records for 100 IDs.
20171213-185627: looking up user records for 100 IDs.
20171213-185628: looking up user records for 100 IDs.
20171213-185629: looking up user records for 100 IDs.
20171213-185631: looking up user records for 100 IDs.
20171213-185632: looking up user records for 100 IDs.
20171213-185633: looking up user records for 100 IDs.
20171213-185634: looking up user records for 100 IDs.
20171213-185635: looking up user records for 100 IDs.
20171213-185637: looking up 

In [8]:
# Now let's write out all the records. I wrote some functions to help.
with open(ofile_name,'w') as ofile :
    write_user_rec_headers(ofile)
    for id, rec in all_records.items() :
        write_user_rec(ofile, rec)